In [91]:
import torch
from torchvision import models
import torch.nn as nn

# 获取alexNet预训练参数
# alexNet = models.alexnet(pretrained=True)
# torch.save(alexNet.state_dict(),'alexNet_weights.pth')
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [92]:
from collections import abc
"""修改 alexnet 使其 iterable"""
class AlexNet(nn.Module):
    def __init__(self,input_layer = 3,num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_layer, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

    def __iter__(self,):
        return SentenceIterator(self.features,self.avgpool,self.classifier)

class SentenceIterator(abc.Iterator):
    def __init__(self,features,avgpool,classifier):
        self.features = features
        self.avg_pool = avgpool
        self.classifier = classifier
        self._index = 0
        self.len1 = len(features)
        self.len2 = 1
        self.len3 = len(classifier)


    def __next__(self):
        try:
            if self._index < self.len1:
                layer = self.features[self._index]
            elif self._index < (self.len1 + self.len2):
                layer = self.avg_pool
            else:
                layer = self.classifier[self._index - self.len1 - self.len2]
        except IndexError:
            raise StopIteration()
        else:
            self._index += 1

        return layer

In [94]:
def alexnet(pretrained: bool = False, **kwargs) -> AlexNet:
    model = AlexNet(**kwargs).to(device)
    if pretrained:
        state_dict = torch.load('./00_model_parameters/alexNet_weights.pth')
        model.load_state_dict(state_dict)
    return model

In [96]:
"""
    使用与训练好的参数先预测一波fashionmnis图像分类的准确率
"""
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

resize = 224
trans = transforms.Compose([
    transforms.Resize(resize),
    transforms.ToTensor(),
])
training_data = datasets.FashionMNIST(root = "../data",train=True,download=False,transform=trans)
test_data = datasets.FashionMNIST(root = "../data",train=False,download=True,transform=trans)

mytest_data = []
for idx,(x,y) in enumerate(test_data):
    mytest_data.append((torch.cat((x,x,x),dim=0),y))

train_dataloader = DataLoader(training_data,batch_size=64,shuffle=True)
test_dataloader = DataLoader(mytest_data,batch_size=64,shuffle=True)

# 预训练最后输出维度1000 所以目前fashion数据集只能多加个 linear 1000->10
# 后续换成cifar-10 数据集可以正常运行
alexNet = nn.Sequential(
    alexnet(pretrained=False,num_classes=1000),
    nn.Linear(1000,10),
).to(device)

In [97]:
"""
    测定test数据集准确率
    batch input [64,1,224,224]
    batch label [64]

    batch output [64,10]
"""
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct,all = 0,0,0
    with torch.no_grad():
        for idx,(X,y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            all += 64
            # X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            if(all % 640 == 0):
                print(f"all number: {all}   correct number: {correct}")
                print(f"Accuracy: {(100*(correct/all)) :> 0.1f}% , avg loss :{test_loss/(idx+1):>8f}")
                print(f"--------------------------------------------------")
    test_loss /= num_batches
    correct /= size
    print(f"test error :\n Accuracy: {(100*correct) :> 0.1f} , avg loss :{test_loss:>8f}")

test(test_dataloader,alexNet,nn.CrossEntropyLoss())

all number: 640   correct number: 63.0
Accuracy:  9.8% , avg loss :2.302670
--------------------------------------------------
all number: 1280   correct number: 125.0
Accuracy:  9.8% , avg loss :2.302659
--------------------------------------------------
all number: 1920   correct number: 176.0
Accuracy:  9.2% , avg loss :2.302793
--------------------------------------------------
all number: 2560   correct number: 242.0
Accuracy:  9.5% , avg loss :2.302923
--------------------------------------------------
all number: 3200   correct number: 297.0
Accuracy:  9.3% , avg loss :2.302683
--------------------------------------------------
all number: 3840   correct number: 357.0
Accuracy:  9.3% , avg loss :2.302678
--------------------------------------------------
all number: 4480   correct number: 423.0
Accuracy:  9.4% , avg loss :2.302618
--------------------------------------------------
all number: 5120   correct number: 486.0
Accuracy:  9.5% , avg loss :2.302684
---------------------

In [99]:
"""
    尝试分解每层的计算
"""
import time
x = torch.rand(size=(10000,3,224,224))
x = x.to(device)
alexNet2 = alexnet(input_layer = 3,num_classes = 1000)

for layer in alexNet2:
    start_time = int(round(time.time() * 1000))
    x = layer(x)
    end_time = int(round(time.time() * 1000))
    print(layer.__class__.__name__,'output shape:\t',x.shape,
          f'\t computation time: {(end_time - start_time)/1000 :>3f} s')

Conv2d output shape:	 torch.Size([10000, 64, 55, 55]) 	 computation time: 30.324000 s
ReLU output shape:	 torch.Size([10000, 64, 55, 55]) 	 computation time: 2.817000 s
MaxPool2d output shape:	 torch.Size([10000, 64, 27, 27]) 	 computation time: 3.487000 s
Conv2d output shape:	 torch.Size([10000, 192, 27, 27]) 	 computation time: 38.240000 s
ReLU output shape:	 torch.Size([10000, 192, 27, 27]) 	 computation time: 0.197000 s
MaxPool2d output shape:	 torch.Size([10000, 192, 13, 13]) 	 computation time: 3.043000 s
Conv2d output shape:	 torch.Size([10000, 384, 13, 13]) 	 computation time: 10.292000 s
ReLU output shape:	 torch.Size([10000, 384, 13, 13]) 	 computation time: 0.407000 s
Conv2d output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 12.459000 s
ReLU output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 0.198000 s
Conv2d output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 7.091000 s
ReLU output shape:	 torch.Size([10000, 256, 13, 13]) 	 

In [74]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
)

print(len(model))

3
